In [1]:
import json
import requests
import os
import urllib.parse

In [2]:
json_str = '{"link":"\/vod\/wuyizhidi\/1-1.html","link_next":null,"link_pre":null,"url":"\/m3u8\/player\/?vod=51509\u0026pid=1\u0026no=1\u0026url=https%3A%2F%2Fikcdn01.ikzybf.com%2F20221008%2FXRC0vaXI%2Findex.m3u8","url_next":null,"poster":"https:\/\/webp.ykjljdcss.com\/upload\/movie\/c\/c6034bb52169e608.webp","adUrl":null,"no":"1","vod":51509}'
json_data = json.loads(json_str)
json_data

{'link': '/vod/wuyizhidi/1-1.html',
 'link_next': None,
 'link_pre': None,
 'url': '/m3u8/player/?vod=51509&pid=1&no=1&url=https%3A%2F%2Fikcdn01.ikzybf.com%2F20221008%2FXRC0vaXI%2Findex.m3u8',
 'url_next': None,
 'poster': 'https://webp.ykjljdcss.com/upload/movie/c/c6034bb52169e608.webp',
 'adUrl': None,
 'no': '1',
 'vod': 51509}

In [3]:
# get the url which is encoded
url_encoded = json_data['url'].split('url=')[-1]
m3u8_url = urllib.parse.unquote(url_encoded)
m3u8_url = urllib.parse.urlparse(m3u8_url)
url_path = m3u8_url.path.split('/')
url_path.insert(-1, '2000kb')
url_path.insert(-1, 'hls')
final_url = urllib.parse.urljoin(m3u8_url.geturl(), '/'.join(url_path)) 
final_url

'https://ikcdn01.ikzybf.com/20221008/XRC0vaXI/2000kb/hls/index.m3u8'

In [5]:
# open a session an request the m3u8 file
session = requests.Session()
headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
    "Host": "ikcdn01.ikzybf.com",
    "Origin": "https://www.7qhb.com",
    "Referer": "https://www.7qhb.com/",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "cross-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "sec-ch-ua": '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
}

response = session.get(final_url, headers=headers)
ts_urls = [urllib.parse.urljoin(final_url, line.strip()) for line in response.text.splitlines() if line and not line.startswith("#")]
ts_urls.__len__(), ts_urls[1]

IndexError: list index out of range

In [ ]:
# we should use the concurrent to 
# write the best downloading program
from concurrent.futures import ThreadPoolExecutor
import ffmpeg
import tempfile
import io

def download_ts_segment(url):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    return response.content

def download_and_process_ts_files(ts_urls):
    with ThreadPoolExecutor(max_workers=10) as executor:
        ts_files = list(executor.map(download_ts_segment, ts_urls))
    return ts_files

def combine_ts_files(ts_files):
    with tempfile.TemporaryDirectory() as tmpdirname:
        print(tmpdirname)
        input_files = []
        for i, ts_file in enumerate(ts_files):
            temp_file_path = f"{tmpdirname}/segment{i}.ts"
            with open(temp_file_path, 'wb') as temp_file:
                temp_file.write(ts_file)
            input_files.append(temp_file_path)
        
        # Create a file containing the list of segment files
        with open(f"{tmpdirname}/file_list.txt", 'w') as file_list:
            for input_file in input_files:
                file_list.write(f"file '{input_file}'\n")
        
        # Combine the segments using ffmpeg
        output_file_path = f"{tmpdirname}/output_movie.mp4"
        ffmpeg.(f"{tmpdirname}/file_list.txt", format='concat', safe=0) \
              .output(output_file_path, vcodec='copy', acodec='copy') \
              .run()
        
        # Read the combined video into memory
        with open(output_file_path, 'rb') as output_file:
            combined_video = output_file.read()
        
        return combined_video

In [ ]:
ts_files = download_and_process_ts_files(ts_urls)
combined_video = combine_ts_files(ts_files)

with open("无主之地.mp4", "wb") as output_file:
    output_file.write(combined_video)

print("Downloaded and combined video saved as output_movie.mp4")

AttributeError: module 'ffmpeg' has no attribute 'input'

In [ ]:
import tempfile
tempfile.TemporaryDirectory()

<TemporaryDirectory 'C:\\Users\\admin\\AppData\\Local\\Temp\\tmp9n6x0_2x'>